In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip3 install imutils
!pip3 install torchsummary
from torchsummary import summary
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from PIL import Image
import imutils
import cv2
from matplotlib import pyplot as plt

  Created wheel for imutils: filename=imutils-0.5.3-py3-none-any.whl size=25850 sha256=83e0dfc4b6fa0b5ef0c25605c27a51cfe141ee351b5fdfde3032d8dc1e438798
  Stored in directory: /root/.cache/pip/wheels/fc/9c/6d/1826267c72afa51b564c9c6e0f66abc806879338bc593a2270
Successfully built imutils


In [3]:
train_data_path = "../input/cat-and-dog/training_set/training_set"
val_data_path = "../input/cat-and-dog/test_set/test_set"

In [4]:
img_transforms = transforms.Compose([
    
    # we resize the pictures
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

img_transforms_train = transforms.Compose([
    
    # we resize the pictures
    transforms.Resize((150,150)),
    transforms.RandomCrop(128),
    transforms.RandomRotation(12),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

train_data = torchvision.datasets.ImageFolder(root=train_data_path,transform=img_transforms_train)

val_data = torchvision.datasets.ImageFolder(root=val_data_path,transform=img_transforms)

In [5]:
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)

val_data_loader  = torch.utils.data.DataLoader(val_data, batch_size=32, shuffle=True)

In [6]:
images,label=next(iter(train_data_loader))


In [7]:
import torch
import numpy as np

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [8]:

import torch.nn as nn
import torch.nn.functional as F

# define the CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # convolutional layer (sees 128x128x3 image tensor)
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        # convolutional layer (sees 64x64x16 tensor)
        self.conv2 = nn.Conv2d(32,64, 3, padding=1)
        # convolutional layer (sees 32x32x32 tensor)
        self.conv3 = nn.Conv2d( 64,128, 3, padding=1)
        # convolutional layer (sees 16x16x64 tensor)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        # max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        # linear layer (256* 8* 8 -> )
        self.fc1 = nn.Linear(256 * 8*8, 8000)
        self.fc2 = nn.Linear(8000, 4000)
        self.fc3 = nn.Linear(4000,2000)
        self.fc4 = nn.Linear(2000, 1000)
        self.fc5 = nn.Linear(1000, 2)
        
        
                
        
        
        
        # dropout layer (p=0.30)
        self.dropout = nn.Dropout(0.30)

    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        # flatten image input
        x = x.view(-1, 256*8* 8)
        # add dropout layer
        x = self.dropout(x)
        # add 1st hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        
        # add 2nd hidden layer, with relu activation function
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)       
        
       
        return x

# create a complete CNN
model = Net()
print(model)

# move tensors to GPU if CUDA is available
if train_on_gpu:
    model.cuda()

Net(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=16384, out_features=8000, bias=True)
  (fc2): Linear(in_features=8000, out_features=4000, bias=True)
  (fc3): Linear(in_features=4000, out_features=2000, bias=True)
  (fc4): Linear(in_features=2000, out_features=1000, bias=True)
  (fc5): Linear(in_features=1000, out_features=2, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)


In [9]:
import torch.optim as optim

# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer
optimizer = optim.Adam(model.parameters(), lr=0.00001)



In [10]:
n_epochs = 30


train,test=[],[]
for i in range(1, n_epochs+1):

    # keep track of training
    
    train_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    for images, label in train_data_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            images, label = images.cuda(), label.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(images)
        # calculate the batch loss
        loss = criterion(output, label)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()
        
    else:
        test_loss = 0
        accuracy = 0
    
        with torch.no_grad():
            model.eval()
            for images,labels in val_data_loader:
                if train_on_gpu:
                    images, labels = images.cuda(), labels.cuda()
                log_ps=model(images)
                test_loss+=criterion(log_ps,labels)


                ps=torch.exp(log_ps)
                top_p,top_class=ps.topk(1,dim=1)
                equals = top_class ==labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))

        train.append(train_loss/len(train_data_loader))
        test.append(test_loss/len(val_data_loader))

        print("epoch: {}/{}..".format(i, n_epochs),
                 "train loss: {:.3f}..".format(train_loss/len(train_data_loader)),
                  "test loss: {:.3f}".format(test_loss/len(val_data_loader)),
                  "test acc: {:.3f}".format(accuracy/len(val_data_loader)))

epoch: 1/30.. train loss: 0.689.. test loss: 0.653 test acc: 0.642
epoch: 2/30.. train loss: 0.644.. test loss: 0.624 test acc: 0.668
epoch: 3/30.. train loss: 0.625.. test loss: 0.619 test acc: 0.663
epoch: 4/30.. train loss: 0.614.. test loss: 0.604 test acc: 0.676
epoch: 5/30.. train loss: 0.598.. test loss: 0.575 test acc: 0.695
epoch: 6/30.. train loss: 0.584.. test loss: 0.558 test acc: 0.710
epoch: 7/30.. train loss: 0.563.. test loss: 0.532 test acc: 0.736
epoch: 8/30.. train loss: 0.540.. test loss: 0.527 test acc: 0.729
epoch: 9/30.. train loss: 0.519.. test loss: 0.511 test acc: 0.748
epoch: 10/30.. train loss: 0.504.. test loss: 0.499 test acc: 0.758
epoch: 11/30.. train loss: 0.499.. test loss: 0.477 test acc: 0.779
epoch: 12/30.. train loss: 0.485.. test loss: 0.482 test acc: 0.762
epoch: 13/30.. train loss: 0.480.. test loss: 0.461 test acc: 0.780
epoch: 14/30.. train loss: 0.470.. test loss: 0.454 test acc: 0.788
epoch: 15/30.. train loss: 0.462.. test loss: 0.467 test 